In [ ]:
date = "20250519"

In [ ]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split,StratifiedKFold

from sklearn.metrics import confusion_matrix,f1_score,classification_report,accuracy_score
from sklearn.metrics import roc_auc_score

from scipy.stats import norm
import matplotlib.pyplot as plt
# import seaborn as sns
import time
import os
import openpyxl

import joblib
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout,Flatten,BatchNormalization
from tensorflow.keras.layers import Conv1D,Input
from tensorflow.keras.layers import MaxPooling1D,UpSampling1D,Lambda,Reshape
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.models import Model


from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU

# from tensorflow.keras.losses import mse
from sklearn.ensemble import RandomForestClassifier

from collections import Counter
from imblearn.over_sampling import SMOTE
# from sklearn.utils import class_weight

# for now()
import datetime
# for timezone()
import pytz
# using now() to get current time
current_time = datetime.datetime.now(pytz.timezone('America/New_York'))
# printing current time in india
print("The current time is :", current_time)

pd.options.mode.chained_assignment = None

np.seterr(divide='ignore', invalid='ignore')

In [ ]:
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

In [ ]:
_celline = ["A549","NCI.H460","SK.N.SH","HT1080"]
threshold = "class_0"

K = 6
max_miss = 1

# # input path
path_input = "../data/mer%s/"%(K)
# output path
path_output = "../results/without_ae/SMOTE/mer%s_%s/"%(K,date)

path_models = path_output+"models/"

os.makedirs(path_output, exist_ok=True)
os.makedirs(path_models,exist_ok=True)

CV = 5
max_len = 5000
segment = [(200,5000)]

_model_type = ["1DCNN","RF","MLP"]
encoding = "rawcounts"

# <font color=blue> Load data</font>
##  <font color=blue> load cell_line, kmer counts table, generate two class training set, 0 as threshold. </font>

In [ ]:
def get_class(dff):
    for i in range(len(dff)):
        if dff.loc[i,Celline] >= 0:
            dff.loc[i,"Class"] = 1
        else:
            dff.loc[i,"Class"] = 0
    print(">= 0 :{}, < 0 :{}".format(len(dff[dff[Celline]>=0]),len(dff[dff[Celline]<0])))
    return dff


def processdata(dff):    
    train = get_class(dff)
    x_train = train.iloc[:,0:-5]
    y_train = train.iloc[:,-1]
    rci = train[["transcript_id","gene_id","length",Celline,"Class"]]
    return x_train,y_train,rci

from collections import defaultdict
from math import sqrt
def evaluate_matrix(cm):
    idx = [0,1]
    columns = ['Sn','Sp','Preci','MCC','F1-score','OA']
    eva = pd.DataFrame(index=idx,columns=columns)
    oa = np.diag(cm).sum()/cm.sum()*100
    for i in range(len(cm)):
        fp = cm.sum(axis=0)[i]-cm[i][i] # sum of column
        fn = cm.sum(axis=1)[i]-cm[i][i] # sum of row
        tp = cm[i][i]
        tn = cm.sum()-(fp+fn+tp)
        if tp == 0:
            sensitivity = 0
            precision = 0
        else:
            sensitivity = tp / (tp+fn)
            precision = tp / (tp+fp)

        specificity = tn / (tn+fp)

        if precision*sensitivity==0:
            f1_score=0
        else:
            f1_score = 2 * precision * sensitivity / (precision + sensitivity)
        if tp*tn-fp*fn == 0:
            mcc = 0
        else:
            mcc = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
        if i==0:
            eva.loc[i] = [sensitivity,specificity,precision,mcc,f1_score,oa]
        else:
            eva.loc[i] = [sensitivity,specificity,precision,mcc,f1_score,0]
    return oa , eva

def build_MLP(input_shape):
    model = Sequential([
        Dense(128, input_shape=(input_shape,)),
        LeakyReLU(alpha=0.01),
#         Dropout(0.2),  # Dropout to avoid overfitting
        Dense(128//2),
        LeakyReLU(alpha=0.01),
        Dropout(0.2),
        Dense(128//4),
        LeakyReLU(alpha=0.01),
        Dropout(0.3),
        Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
    ])
    # compile the keras model
    model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])
#     model.summary()
    return model

def build_CNN(input_shape):
    model = Sequential([
        Conv1D(filters=128, kernel_size=3, activation='relu',kernel_initializer=HeNormal(), input_shape=(input_shape, 1)),
        MaxPooling1D(pool_size=3),
        Conv1D(filters=256, kernel_size=3, activation='relu',kernel_initializer=HeNormal()),
        MaxPooling1D(pool_size=3),
        Flatten(),
        Dense(256, activation='relu', kernel_initializer=HeNormal()),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
#     cnn1.summary()
    return model

############################################################################################################
def trainModel(n_fold,m,n,model,model_types,x_train,x_val,y_train, y_val):

    early_stopping = EarlyStopping(monitor='val_loss', mode="min",patience=10,verbose=0)
    model_checkpoint = ModelCheckpoint(path_models+'%s_%s_%s_%s_%smer%smiss_fold%s.h5'%(encoding,model_types,m,n,K,Q,n_fold), monitor='val_loss',
                                       patience=5, verbose=0, save_best_only=True,save_weights_only=True)

    # fit the keras model on the dataset
    history = model.fit(x_train, y_train, epochs=30,
                        batch_size=64,
                        verbose=0,
                        callbacks=[early_stopping,model_checkpoint],
                        validation_data=(x_val, y_val))
    # evaluate the keras model
    model.load_weights(path_models+'%s_%s_%s_%s_%smer%smiss_fold%s.h5'%(encoding,model_types,m,n,K,Q,n_fold))
    score = model.evaluate(x_val, y_val)
#     print("val accuracy: ",score[1]*100)
    return model,score[1]

def evaluateModel(model_types,model,x_val,y_val,dff_val):
    if model_types == "RF":
        y_pred_prob = model.predict_proba(x_val)[:, 1] 
    else:
        y_pred_prob = model.predict(x_val).ravel() 
    auc = roc_auc_score(y_val, y_pred_prob)
    
    y_pred = (model.predict(x_val) > 0.5).astype("int32")    
    accuracy = accuracy_score(y_val, y_pred)
    
    cm = confusion_matrix(y_val, y_pred)
    oa, eva = evaluate_matrix(cm)
    eva.loc[0,"AUC"] = auc
    dff_val = pd.concat([dff_val,pd.DataFrame(y_pred,columns=["pred"])],axis=1)
    dff_val = pd.concat([dff_val,pd.DataFrame(y_pred_prob,columns=["pred_prob"])],axis=1)

    return oa, cm, eva,dff_val,auc

In [ ]:
def train():

    # COUNT Running time
    start = time.time()
    wb = openpyxl.Workbook()
    
    for model_type in _model_type:
        for (m,n) in segment:           
            eva_val0_file = path_output + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_val0.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
            predict_file_val = path_output + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_val.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
            eva_test0_file = path_output + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test0.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
            predict_file_test = path_output + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_test.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
#             eva_test_apex_file = path_apex + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test_apex.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
#             predict_file_apex = path_apex + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_apex.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)

            wb.save(eva_val0_file)
            wb.save(predict_file_val)
            wb.save(eva_test0_file)
            wb.save(predict_file_test)
#             wb.save(eva_test_apex_file)
#             wb.save(predict_file_apex)

    for j in range(0,2):

        # training dataset split by genes, 5-fold CV
        kf = KFold(n_splits=CV,random_state=(j+1)*42, shuffle=True)
        i=0

        for train_index, val_index in kf.split(genes):
            i += 1
            print("{} No.{} fold".format(Celline,j*5+i))

            for (m,n) in segment:

                train_genes = genes.iloc[train_index].gene_id
                val_genes = genes.iloc[val_index].gene_id
                df_train = _df_train.copy()   ## here
                df_test = _df_test.copy()
#                 df_apex = _df_apex.copy()

                train = df_train[df_train.gene_id.isin(train_genes)]
                train = train.sample(frac=1).reset_index(drop=True)
                x_train,y_train,train_rci = processdata(train)
                x_train = x_train.div(train_rci["length"]-K+1,axis=0)
        
                count = Counter(y_train)
                ratio = count[0]/count[1]
                print("class 0 : class 1 = %.3f."%(ratio))
                smt = SMOTE(sampling_strategy='minority', random_state=42)
                x_train, y_train = smt.fit_resample(x_train, y_train)
                print("After over-sampling",(Counter(y_train)))

                val = df_train[df_train.gene_id.isin(val_genes)].reset_index(drop=True)
                x_val,y_val,val_rci = processdata(val)
                x_val = x_val.div(val_rci["length"]-K+1,axis=0)

                test = df_test.copy()
                x_test,y_test,test_rci = processdata(test)
                x_test = x_test.div(test_rci["length"]-K+1,axis=0)
                
                x_train = x_train.to_numpy()
                x_val = x_val.to_numpy()
                x_test = x_test.to_numpy()

#                 x_test_apex = _df_apex.iloc[:,0:-4]
#                 y_test_apex = _df_apex.iloc[:,-1]
#                 apex_pre = _df_apex[["transcript_id","gene_id","length","Class"]]

                input_dim = x_train.shape[1]
                for model_type in _model_type:
                    test_set_type = "lncATLAS"

                    print()
                    ### load model
                    print("Training on  %s %smer%smismatch %s %s %s %s. "%(Celline,K,Q,encoding, model_type,m,n))
                    print()
                    if model_type == "1DCNN":
                        k = x_train.shape[1]
                        model = build_CNN(x_train.shape[1])
                        x_train_cnn = x_train.reshape(len(x_train),k,1)
                        x_val0_cnn = x_val.reshape(len(x_val),k,1)
                        x_test0_cnn = x_test.reshape(len(x_test),k,1)
#                         x_test_apex_cnn = x_test_apex.reshape(len(x_test_apex),k,1)

                        model,score = trainModel(j*5+i,m,n,model,model_type,x_train_cnn,x_val0_cnn,y_train, y_val)

                        oa_val, cm_val, eva_val, val0_predict,auc_val = evaluateModel(model_type,model,x_val0_cnn,y_val,val_rci)
                        oa_test, cm_test,eva_test,test0_predict,auc_test = evaluateModel(model_type,model,x_test0_cnn,y_test,test_rci)
#                         test_apex_eva,test_apex_len_acc,test_apex_report,test_apex_predict = evaluateModel("apex",model_type,model,x_test_apex_cnn,y_test_apex_cnn,apex_pre)
                        tf.keras.backend.clear_session()
                        gc.collect() # Run garbage collection

                    elif model_type == "MLP":
                        model = build_MLP(x_train.shape[1])
                        model,score = trainModel(j*5+i,m,n,model,model_type,x_train,x_val,y_train, y_val)

                        oa_val, cm_val, eva_val, val0_predict,auc_val = evaluateModel(model_type,model,x_val,y_val,val_rci)
                        oa_test, cm_test,eva_test,test0_predict,auc_test = evaluateModel(model_type,model,x_test,y_test,test_rci)
#                         test_apex_eva,test_apex_len_acc,test_apex_report,test_apex_predict = evaluateModel("apex",model_type,model,x_test_apex,y_test_apex_cnn,apex_pre)
                        tf.keras.backend.clear_session()
                        gc.collect() # Run garbage collection
        
                    else:
                        model_type == "RF"
                        model = RandomForestClassifier(n_estimators=100,n_jobs=-1)
                        model.fit(x_train,y_train)
                        joblib.dump(model,path_models+'%s_%s_%s_%s_%smer%smiss_fold%s.h5'%(encoding,model_type,m,n,K,Q,j*5+i))
                        y_predit0 = model.predict(x_val)
                        score = accuracy_score(y_val,y_predit0)
                        print("RF accuracy:",score)
                        print()
                        oa_val, cm_val, eva_val, val0_predict,auc_val = evaluateModel(model_type,model,x_val,y_val,val_rci)
                        oa_test, cm_test,eva_test,test0_predict,auc_test = evaluateModel(model_type,model,x_test,y_test,test_rci)
#                         test_apex_eva,test_apex_len_acc,test_apex_report,test_apex_predict = evaluateModel("apex",model_type,model,x_test_apex,y_test_apex,apex_pre)
                        del model  # Explicitly delete the model
                        gc.collect()  # Run garbage collection

                    ## validation class threshold 0 and mean+/-std
                    print(f'model {model_type} val accu {oa_val:.3f}, auc {auc_val:.3f}\nval cm {cm_val}')
#                     print(f'model {model_type} test accu {oa_test:.3f}, auc {auc_test:.3f}\ntest cm {cm_test}')
            
                    eva_val0_file = path_output + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_val0.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
                    predict_file_val = path_output + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_val.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
            
                    writer_eva_val0 = ExcelWriter(eva_val0_file, engine='openpyxl',mode='a')
                    # writer_eva_valstd = ExcelWriter(eva_valstd_file, engine='openpyxl',mode='a')
                    writer_predict_val = ExcelWriter(predict_file_val, engine='openpyxl',mode='a')
                    
                    conf_matrix_df = pd.DataFrame(cm_val, columns=['Pred_0', 'Pred_1'], index=['True_0', 'True_1'])    

                    eva_val.to_excel(writer_eva_val0,sheet_name="eva{}".format(j*5+i))
                    conf_matrix_df.to_excel(writer_eva_val0,sheet_name="CM{}".format(j*5+i))                    
                    val0_predict.to_excel(writer_predict_val,sheet_name="predict_0_{}".format(j*5+i))

                    writer_eva_val0.close()
                    writer_predict_val.close()


                    ## prediction on test,threshold 0 and mean+/-std


                    eva_test0_file = path_output + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test0.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
                    predict_file_test = path_output + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_test.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)

                    writer_eva_test0 = ExcelWriter(eva_test0_file, engine='openpyxl',mode='a')
                    writer_predict_test = ExcelWriter(predict_file_test, engine='openpyxl',mode='a')
                    
                    conf_matrix_df = pd.DataFrame(cm_test, columns=['Pred_0', 'Pred_1'], index=['True_0', 'True_1'])  

                    eva_test.to_excel(writer_eva_test0,sheet_name="eva{}".format(j*5+i))
                    conf_matrix_df.to_excel(writer_eva_test0,sheet_name="CM{}".format(j*5+i))
                    test0_predict.to_excel(writer_predict_test,sheet_name="predict_0_{}".format(j*5+i))                  

                    writer_eva_test0.close()
                    writer_predict_test.close()


#                     eva_test_apex_file = path_apex + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test_apex.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
#                     predict_file_apex = path_apex + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_apex.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)

#                     writer_eva_test_apex = ExcelWriter(eva_test_apex_file, engine='openpyxl',mode='a')
#                     writer_predict_test_apex = ExcelWriter(predict_file_apex, engine='openpyxl',mode='a')

#                     test_apex_eva.to_excel(writer_eva_test_apex,sheet_name="eva{}".format(j*5+i))
#                     test_apex_len_acc.to_excel(writer_eva_test_apex,sheet_name="len_acc{}".format(j*5+i))
#                     test_apex_report.to_excel(writer_eva_test_apex,sheet_name="report{}".format(j*5+i))
#                     test_apex_predict.to_excel(writer_predict_test_apex,sheet_name="predict_0_{}".format(j*5+i))  

#                     writer_eva_test_apex.close()
#                     writer_predict_test_apex.close()                       
                        

    end = time.time()
    print("Running time {} seconds".format(round(end-start,3)))

## <font color="blue"> only validation </font>

In [ ]:
for Q in range(0,max_miss+1):
    _train = pd.read_csv(path_input+encoding+"_noncoding_train_%smer%smiss.csv"%(K,Q),index_col=0)
    data_train = _train.iloc[:,0:-18]
    data_train[["transcript_id","gene_id","length"]] = _train[["transcript_id","gene_id","length"]]

    _test = pd.read_csv(path_input+encoding+"_noncoding_test_%smer%smiss.csv"%(K,Q),index_col=0)
    data_test = _test.iloc[:,0:-18]
    data_test[["transcript_id","gene_id","length"]] = _test[["transcript_id","gene_id","length"]]

#     _apex = pd.read_csv(apex_seq+encoding+"_coding_apex_%smer%smiss_200-5000_longest.csv"%(K,Q),index_col=0)
#     data_apex = _apex.iloc[:,0:-6].div(_apex["length"],axis=0)
#     data_apex[["transcript_id","gene_id","length","Class"]] =  _apex[["transcript_id","gene_id","length","Class"]]


    for Celline in _celline:
        _df_train = data_train.copy()
        _df_train[Celline] = _train[Celline]
        _df_train = _df_train.dropna(axis=0).reset_index(drop=True)


        _df_test = data_test.copy()
        _df_test[Celline] = _test[Celline]
        _df_test = _df_test.dropna(axis=0).reset_index(drop=True)

#         _df_apex = data_apex.copy()
#         _df_apex["Class"] = _apex["Class"]
#         _df_apex = _df_apex.dropna(axis=0).reset_index(drop=True)

        genes = pd.DataFrame(_df_train.gene_id.unique(),columns=["gene_id"])

        print()
        print("training on %s %smer%smismatch ."%(Celline,K,Q ))
        

        train()

In [ ]:
for model_type in _model_type:
    for (m,n) in segment:
        for Celline in _celline:
            for Q in range(0,max_miss+1):
                
                eva_val0_file = path_output + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_val0.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
                eva_test0_file = path_output + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test0.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
#                 eva_test_apex_file = path_apex + "10fold_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test_apex.xlsx"%(Celline,m,n,encoding,threshold,model_type,K,Q)
                for infile in [eva_val0_file,eva_test0_file]:
                    print(infile)
                    writer = ExcelWriter(infile, engine='openpyxl',mode='a')
                    xl = pd.ExcelFile(infile)
                    eva = []

                    for i in range(1,11):
                        eva.append(xl.parse('eva'+str(i),index_col=0))

                    df_eva_avg =  pd.concat(eva).groupby(level=0).mean()

                    df_eva_avg.to_excel(writer,sheet_name='eva_avg')

                    writer.close()

In [ ]:
current_time = datetime.datetime.now(pytz.timezone('America/New_York'))
# printing current time in india
print("The current time is :", current_time)